### IMPORTING LIBRARIES

In [1]:
import numpy as np
import pandas as pd
import string

### READING THE DATA

In [2]:
question= pd.read_csv('E:/Avishkar Tasks/Stack overflow tag prediction/Questions.csv', encoding='latin')
answer= pd.read_csv('E:/Avishkar Tasks/Stack overflow tag prediction/Answers.csv', encoding='latin')
tags= pd.read_csv('E:/Avishkar Tasks/Stack overflow tag prediction/Tags.csv', encoding='latin')

In [3]:
question.head()

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body
0,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...
1,90,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...
2,120,83.0,2008-08-01T15:50:08Z,NaN,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...
3,180,2089740.0,2008-08-01T18:42:19Z,NaN,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...
4,260,91.0,2008-08-01T23:22:08Z,NaN,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...


In [4]:
answer.head()

,Id,OwnerUserId,CreationDate,ParentId,Score,Body
0,92,61.0,2008-08-01T14:45:37Z,90,13,"<p><a href=""http://svnbook.red-bean.com/"">Vers..."
1,124,26.0,2008-08-01T16:09:47Z,80,12,<p>I wound up using this. It is a kind of a ha...
2,199,50.0,2008-08-01T19:36:46Z,180,1,<p>I've read somewhere the human eye can't dis...
3,269,91.0,2008-08-01T23:49:57Z,260,4,"<p>Yes, I thought about that, but I soon figur..."
4,307,49.0,2008-08-02T01:49:46Z,260,28,"<p><a href=""http://www.codeproject.com/Article..."


In [5]:
tags.head()

,Id,Tag
0,80,flex
1,80,actionscript-3
2,80,air
3,90,svn
4,90,tortoisesvn


In [6]:
print(question.shape, answer.shape, tags.shape)

(1264216, 7) (2014516, 6) (3750994, 2)


In [7]:
print(question.Id.nunique(), answer.ParentId.nunique(), tags.Id.nunique())

1264216 1102568 1264216


Number of rows are different in the dataframes. 
The ids have been repeated in the tags & answer dataframe, because of the discrepancy between the number of rows and the unique IDs.

### MERGING THE DATA FRAMES

In [8]:
answer.drop(columns=['Id','OwnerUserId', 'CreationDate'],inplace=True)
answer.columns=['Id', 'A_Score', 'A_Body']

In [9]:
grouped_answer = answer.groupby("Id")['A_Body'].apply(lambda answer: ' '.join(answer))
grouped_answer= grouped_answer.to_frame()
grouped_answer= grouped_answer.sort_values(by='Id')
grouped_answer.head()

,A_Body
Id,
80,<p>I wound up using this. It is a kind of a ha...
90,"<p><a href=""http://svnbook.red-bean.com/"">Vers..."
120,<p>The Jeff Prosise version from MSDN magazine...
180,<p>I've read somewhere the human eye can't dis...
260,"<p>Yes, I thought about that, but I soon figur..."


In [10]:
tags['Tag']= tags['Tag'].astype(str)
grouped_tags = tags.groupby("Id")['Tag'].apply(lambda tags: ' '.join(tags))
#grouped_tags = tags.groupby("Id")['Tag'].apply(lambda tags: ' '.join(tags))

grouped_tags= grouped_tags.to_frame()
grouped_tags= grouped_tags.sort_values(by='Id')
grouped_tags.head()

,Tag
Id,
80,flex actionscript-3 air
90,svn tortoisesvn branch branching-and-merging
120,sql asp.net sitemap
180,algorithm language-agnostic colors color-space
260,c# .net scripting compiler-construction


In [11]:
print(grouped_answer.shape, grouped_tags.shape)

(1102568, 1) (1264216, 1)


1. Merging Question and grouped_answer dataframes to get df
2. Merging df and grouped_answer dataframes to get df1

In [12]:
grouped_answer['Ids']= grouped_answer.index
grouped_tags['Ids']= grouped_tags.index
question.columns= ['Ids', 'OwnerUserId', 'CreationDate', 'ClosedDate', 'Score', 'Title',
       'Body']
question= question.sort_values(by='Ids')
df= pd.merge(question,grouped_answer,how='left')
df1= pd.merge(df,grouped_tags,how='left',on='Ids')

In [13]:
df1.head()

,Ids,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body,A_Body,Tag
0,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...,<p>I wound up using this. It is a kind of a ha...,flex actionscript-3 air
1,90,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...,"<p><a href=""http://svnbook.red-bean.com/"">Vers...",svn tortoisesvn branch branching-and-merging
2,120,83.0,2008-08-01T15:50:08Z,NaN,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...,<p>The Jeff Prosise version from MSDN magazine...,sql asp.net sitemap
3,180,2089740.0,2008-08-01T18:42:19Z,NaN,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...,<p>I've read somewhere the human eye can't dis...,algorithm language-agnostic colors color-space
4,260,91.0,2008-08-01T23:22:08Z,NaN,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...,"<p>Yes, I thought about that, but I soon figur...",c# .net scripting compiler-construction


### REMOVING UNNECESSARY VARIABLES

In [14]:
df1.drop(columns=['Ids', 'OwnerUserId', 'CreationDate', 'ClosedDate'],inplace=True)
df1.head()

,Score,Title,Body,A_Body,Tag
0,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...,<p>I wound up using this. It is a kind of a ha...,flex actionscript-3 air
1,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...,"<p><a href=""http://svnbook.red-bean.com/"">Vers...",svn tortoisesvn branch branching-and-merging
2,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...,<p>The Jeff Prosise version from MSDN magazine...,sql asp.net sitemap
3,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...,<p>I've read somewhere the human eye can't dis...,algorithm language-agnostic colors color-space
4,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...,"<p>Yes, I thought about that, but I soon figur...",c# .net scripting compiler-construction


### CHECKING FOR DUPLICATE ROWS

In [15]:
df1=df1.drop_duplicates()
df1.shape

(1264216, 5)

There are no duplicate rows because the number of rows after applying drop_duplicates function, remains the same. 

### FILTERING DATA BASED ON SCORE AND MOST FREQUENTLY USED TAGS

In [16]:
print(df1.Score.min(), df1.Score.max())

-73 5190


In [17]:
z= df1['Tag'].value_counts().sort_values(ascending=False)
z.index

Index(['android', 'java', 'php', 'javascript jquery', 'javascript', 'jquery',
       'python', 'html css', 'c#', 'php mysql',
       ...
       'c# asp.net .net visual-studio-2010 visual-studio',
       'scala playframework-2.0 elasticsearch', 'html css jquery-plugins',
       'python distributed-computing dask', 'oop strategy-pattern',
       'android eclipse sdk adb avd', 'soap wso2 wso2esb wso2carbon',
       'javascript math colors color-theory', 'css html5 css3 vendor-prefix',
       'java android reflection override'],
      dtype='object', length=685700)

In [18]:
df2= df1.groupby(by='Tag')['Tag'].count().sort_values(ascending=False).to_frame()
df2.columns= ['Tag_count']
df2['Tags']=df2.index

In [20]:
df1.columns= ['Score', 'Title', 'Body', 'A_Body', 'Tags']
df1= pd.merge(df1,df2,how='left',on='Tags')
df1.head()

,Score,Title,Body,A_Body,Tags,Tag_count
0,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...,<p>I wound up using this. It is a kind of a ha...,flex actionscript-3 air,11
1,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...,"<p><a href=""http://svnbook.red-bean.com/"">Vers...",svn tortoisesvn branch branching-and-merging,1
2,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...,<p>The Jeff Prosise version from MSDN magazine...,sql asp.net sitemap,1
3,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...,<p>I've read somewhere the human eye can't dis...,algorithm language-agnostic colors color-space,1
4,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...,"<p>Yes, I thought about that, but I soon figur...",c# .net scripting compiler-construction,1


In [21]:
df1= df1[df1['Tag_count']>=1000]
df1= df1[df1['Score']>3]
df1.shape

(7654, 6)

For better prediction we will be using only those tags which have been repeated for atleast 1000 times and the score is more than 3. Low scores mean that the question is either erroneous or does not have sufficient information.

In [22]:
df1.Tags.value_counts().sort_values(ascending=False)

android                       738
javascript                    513
java                          493
python                        491
javascript jquery             429
c#                            413
c++                           402
php                           342
jquery                        327
html css                      292
git                           281
r                             264
c                             217
mysql                         187
ruby                          170
css                           166
angularjs                     146
javascript angularjs          142
ruby-on-rails                 129
java android                  118
ruby-on-rails ruby            118
php mysql                     117
javascript jquery html        114
sql sql-server                113
python django                 100
c# asp.net                     96
javascript jquery html css     90
javascript html                89
mysql sql                      89
iphone        

### CHECKING FOR MISSING VALUES

In [23]:
print(df1.isnull().sum())

print('Shape of df1:',df1.shape)

Score         0
Title         0
Body          0
A_Body       65
Tags          0
Tag_count     0
dtype: int64
Shape of df1: (7654, 6)


A_Body is neither categorical nor continuous in nature. So it is not possible to impute its missing values. So we will remove it from our analysis

In [24]:
df.drop(columns=['A_Body'],inplace=True)

### CLEANING THE TEXT FOR TITLE AND BODY

1. Removing punctuation
2. Removing HTML tags (if required)
3. Changing text into lowercase
4. Splitting the text into words
5. Removing stopwords

#### PUNCTUATION & HTML TAGS REMOVAL, LOWERCASE, WORD TOKENIZATION

In [25]:
def remove_punctuation(text):
    for punctuation in string.punctuation:
        text= text.replace(punctuation,'')
    return text
    

In [26]:
df1['Title']= df1['Title'].astype(str)

df1['Title1']= df1['Title'].apply(remove_punctuation)
df1['Title1']=df1['Title1'].str.lower()
df1['Title1']= df1['Title1'].str.split()
df1['Title1'].head()

11    [how, to, get, the, value, of, built, encoded,...
19    [can, i, logically, reorder, columns, in, a, t...
23                    [convert, hashbytes, to, varchar]
34          [mysqlapache, error, in, php, mysql, query]
48    [the, difference, between, a, datagrid, and, a...
Name: Title1, dtype: object

In [27]:
df1['Body']= df1['Body'].astype(str)
import re

df1['Body1']= df1['Body'].apply(lambda x: re.sub('<[^<]+?>','',x))
df1['Body1'].head()

11    I need to grab the base64-encoded representati...
19    If I'm adding a column to a table in Microsoft...
23    I want to get the MD5 Hash of a string value i...
34    I am getting the following error:\n\n\n  Acces...
48    I've been doing ASP.NET development for a litt...
Name: Body1, dtype: object

In [28]:
df1['Body1']= df1['Body1'].apply(remove_punctuation)
df1['Body1']=df1['Body1'].str.lower()
df1['Body1']= df1['Body1'].str.split()
df1['Body1'].head()

11    [i, need, to, grab, the, base64encoded, repres...
19    [if, im, adding, a, column, to, a, table, in, ...
23    [i, want, to, get, the, md5, hash, of, a, stri...
34    [i, am, getting, the, following, error, access...
48    [ive, been, doing, aspnet, development, for, a...
Name: Body1, dtype: object

#### LEMMATIZATION

In [29]:
from nltk.stem import WordNetLemmatizer
lematizer= WordNetLemmatizer()

def word_lemmatizer(text):
    lem_text=[lematizer.lemmatize(i) for i in text]
    return lem_text

In [30]:
df1['Title1']= df1['Title1'].apply(lambda x: word_lemmatizer(x))

In [31]:
df1['Body1']= df1['Body1'].apply(lambda x: word_lemmatizer(x))

#### STOPWORD REMOVAL USING SPACY

In [32]:
import spacy
sp= spacy.load('en_core_web_sm')
all_stopwords= sp.Defaults.stop_words
df1['Title1']= df1['Title1'].apply(lambda x:[word for word in x if not word in all_stopwords])
df1['Title1'].head()                                   


11          [value, built, encoded, viewstate]
19         [logically, reorder, column, table]
23               [convert, hashbytes, varchar]
34     [mysqlapache, error, php, mysql, query]
48    [difference, datagrid, gridview, aspnet]
Name: Title1, dtype: object

In [33]:
import spacy
sp= spacy.load('en_core_web_sm')
all_stopwords= sp.Defaults.stop_words
df1['Body1']= df1['Body1'].apply(lambda x:[word for word in x if not word in all_stopwords])
df1['Body1'].head()                                   

11    [need, grab, base64encoded, representation, vi...
19    [im, adding, column, table, microsoft, sql, se...
23    [want, md5, hash, string, value, sql, server, ...
34    [getting, following, error, access, denied, us...
48    [ive, aspnet, development, little, ive, gridvi...
Name: Body1, dtype: object

### FINAL DATAFRAME AFTER TEXT CLEANING

In [34]:
df1.drop(columns=['Title', 'Body', 'Tag_count','Score'], inplace=True)
df1.head()

,A_Body,Tags,Title1,Body1
11,"<p>Rex, I suspect a good place to start lookin...",c# asp.net,"[value, built, encoded, viewstate]","[need, grab, base64encoded, representation, vi..."
19,<P>You can not do this programatically (in a s...,sql-server,"[logically, reorder, column, table]","[im, adding, column, table, microsoft, sql, se..."
23,<p>I have found the solution else where:</p>\r...,sql sql-server,"[convert, hashbytes, varchar]","[want, md5, hash, string, value, sql, server, ..."
34,"<p>Just to check, if you use <strong>just</str...",php mysql,"[mysqlapache, error, php, mysql, query]","[getting, following, error, access, denied, us..."
48,<p>The DataGrid was originally in .NET 1.0. T...,asp.net,"[difference, datagrid, gridview, aspnet]","[ive, aspnet, development, little, ive, gridvi..."


### TF-IDF VECTORIZATION

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer

df1['Title1']= df1['Title1'].astype(str)
vectorizer = TfidfVectorizer()
X1 = vectorizer.fit_transform(df1['Title1'].str.lower())

In [36]:
df1['Body1']= df1['Body1'].astype(str)
vectorizer = TfidfVectorizer()
X2 = vectorizer.fit_transform(df1['Body1'].str.lower())

### CHANGING CATEGORICAL VARIABLES INTO NUMERIC

In [37]:
from sklearn.preprocessing import LabelEncoder
le= LabelEncoder() 
df1['Tags']= le.fit_transform(df1['Tags'])

### SPLITTING THE DATASET INTO TRAIN AND TEST SET

In [38]:
y = df1['Tags'].values

In [39]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X2, y, test_size=0.30, random_state=42)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(5357, 65095) (2297, 65095) (5357,) (2297,)


### APPLYING DIFFERENT ALGORITHMS

In [40]:
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

C:\Users\AMRITA\Anaconda3\lib\site-packages\dask\dataframe\utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
C:\Users\AMRITA\Anaconda3\lib\site-packages\distributed\utils.py:133: RuntimeWarning: Couldn't detect a suitable IP address for reaching '8.8.8.8', defaulting to '127.0.0.1': [WinError 10065] A socket operation was attempted to an unreachable host
  RuntimeWarning,


In [41]:
clf = LogisticRegression(C=10)

# Creating the model on Training Data
LOG=clf.fit(x_train,y_train)
prediction=LOG.predict(x_test)

# Printing the Overall Accuracy of the model
from sklearn import metrics
print(metrics.confusion_matrix(y_test, prediction))
F1_Score=metrics.classification_report(y_test, prediction).split()[-2]
print('Accuracy of the model:', F1_Score)

C:\Users\AMRITA\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[[196   0   0 ...   0   0   0]
 [  3   5   0 ...   0   0   0]
 [  3   0  13 ...   0   0   0]
 ...
 [  2   0   0 ...  10   0   0]
 [  1   0   0 ...   8   0   0]
 [  2   0   0 ...   0   0   0]]
Accuracy of the model: 0.54


C:\Users\AMRITA\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [42]:
clf=XGBClassifier(max_depth=2, learning_rate=0.2, n_estimators=400, objective='binary:logistic', booster='gbtree')

# Creating the model on Training Data
XGB=clf.fit(x_train,y_train)
prediction=XGB.predict(x_test)

# Measuring accuracy on Testing Data
from sklearn import metrics
print(metrics.confusion_matrix(y_test, prediction))

# Printing the Overall Accuracy of the model
F1_Score=metrics.classification_report(y_test, prediction).split()[-2]
print('Accuracy of the model:', F1_Score)

[[178   0   0 ...   0   0   0]
 [  0  19   0 ...   0   0   0]
 [  2   0  13 ...   0   0   0]
 ...
 [  1   0   0 ...  10   4   0]
 [  1   0   0 ...   5   3   0]
 [  1   0   0 ...   0   0   8]]
Accuracy of the model: 0.55


In [43]:
model = MultinomialNB().fit(x_train,y_train)
prediction= model.predict(x_test)

from sklearn import metrics
print(metrics.confusion_matrix(y_test, prediction))

# Printing the Overall Accuracy of the model
F1_Score=metrics.classification_report(y_test, prediction).split()[-2]
print('Accuracy of the model:', F1_Score)

[[202   0   0 ...   0   0   0]
 [ 29   0   0 ...   0   0   0]
 [ 27   0   0 ...   0   0   0]
 ...
 [ 17   0   0 ...   0   0   0]
 [ 14   0   0 ...   0   0   0]
 [ 13   0   0 ...   0   0   0]]
Accuracy of the model: 0.24


C:\Users\AMRITA\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [44]:
clf = KNeighborsClassifier(n_neighbors=4)

# Creating the model on Training Data
KNN=clf.fit(x_train,y_train)
prediction=KNN.predict(x_test)

# Measuring accuracy on Testing Data
from sklearn import metrics
print(metrics.confusion_matrix(y_test, prediction))

# Printing the Overall Accuracy of the model
F1_Score=metrics.classification_report(y_test, prediction).split()[-2]
print('Accuracy of the model:', F1_Score)


[[165   2   1 ...   0   0   0]
 [  6   4   2 ...   0   0   0]
 [  3   1  10 ...   0   0   0]
 ...
 [  2   1   1 ...   1   0   0]
 [  3   0   0 ...   2   1   0]
 [  2   0   1 ...   0   0   0]]
Accuracy of the model: 0.30


C:\Users\AMRITA\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [45]:
clf = RandomForestClassifier(max_depth=4, n_estimators=600,criterion='entropy')

# Creating the model on Training Data
RF=clf.fit(x_train,y_train)
prediction=RF.predict(x_test)

# Measuring accuracy on Testing Data
from sklearn import metrics
print(metrics.confusion_matrix(y_test, prediction))

# Printing the Overall Accuracy of the model
F1_Score=metrics.classification_report(y_test, prediction).split()[-2]
print('Accuracy of the model:', F1_Score)

[[203   0   0 ...   0   0   0]
 [ 47   0   0 ...   0   0   0]
 [ 29   0   0 ...   0   0   0]
 ...
 [ 20   0   0 ...   0   0   0]
 [ 14   0   0 ...   0   0   0]
 [ 16   0   0 ...   0   0   0]]
Accuracy of the model: 0.05


C:\Users\AMRITA\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


__Applied the ML algorithms on the TF-IDF vectorization of body because it is giving a higher accuracy as compared to the TF-IDF vectorization of the title.__

__XGBoost is giving the highest accuracy.__

__F1 Score: 55%__